In [6]:
from elasticsearch import Elasticsearch
import yaml
import json

In [7]:
def get_config():
    with open('fastapi.yml', encoding='utf-8') as reader:
        conf = yaml.safe_load(reader)
    
    es_conf = conf.get("ELASTICSEARCH")

    return es_conf

In [8]:
conf = get_config()
es = Elasticsearch(**conf['SESSION_KWARGS'])

In [9]:
def print_geoextent(extent):
    cords = extent['coordinates']
    print(f'   {cords[0][1]:.2f}\t')
    print(f'{cords[0][0]:.2f}\t{cords[1][0]:.2f}')
    print(f'   {cords[1][1]:.2f}\t')


In [10]:
def print_record(record):
    print(f'Record_type: {record["model"]}')
    print(f'Title: {record["title"]}')
    print(f'Keywords: {record["keywords"]}')
    print(f'Abstract: {record["abstract"][:100]}...')
    print_geoextent(record['geographicExtent'])
    print()

In [11]:
import collections

def generate_dict(dep=1):
    if dep == 1:
        return collections.defaultdict(dict)
    return collections.defaultdict(lambda: generate_dict(dep - 1))

def filter_by_record_type(record_type = '', bbox = '', bbox_relation = 'intersect'):
    # fancy thing found on stackoverflow
    my_query = generate_dict(7)

    if record_type:
        my_query['query']['bool']['must']['terms']['model'] = record_type

    if bbox:
        my_query['query']['bool']['filter']['geo_shape']['geographicExtent']['shape']['type'] = 'envelope'
        my_query['query']['bool']['filter']['geo_shape']['geographicExtent']['shape']['coordinates'] = bbox
        my_query['query']['bool']['filter']['geo_shape']['geographicExtent']['relation'] = bbox_relation
        
    return f"{json.dumps(my_query)}"

In [12]:
my_query = filter_by_record_type(record_type=['observation'], bbox = [[-8, 59], [2, 49]], bbox_relation='within')
my_query
hits = es.search(index="stac-moles-test", body=my_query)

for i in hits['hits']['hits']:
    fields = i['_source']
    print_record(fields)


Record_type: observation
Title: Daily Mean, Minimum and Maximum Central England Temperature series
Keywords: HadCET, temperature, Hadley, Met Office
Abstract: The longest available instrumental record of temperature in the world is now available at the BADC. ...
   54.00	
-4.00	1.00
   51.00	

Record_type: observation
Title: Flood Risk for Extreme Events (FREE): Wind profiler measurements from UK stations (June 2007-June 2009)
Keywords: FREE, Wind Profiles
Abstract: The Exploitation of new data sources, data assimilation and ensemble techniques for storm and flood ...
   54.10	
-5.30	1.00
   50.20	

Record_type: observation
Title: Flood Risk for Extreme Events (FREE): Model equivalent (radiosonde, radar, windprofiler) profiles from the UKMO UK 4km operational archives (June 2007-2009)
Keywords: FREE, Radiosonde, Radar, Windprofiler
Abstract: The Exploitation of new data sources, data assimilation and ensemble techniques for storm and flood ...
   55.02	
-1.25	0.32
   51.20	

Record_typ

In [21]:
hits['hits']['hits'][0]['_source']['responsiblePartyInfo']

{'priority': 1,
 'party': {'firstName': '',
  'lastName': 'NCAS British Atmospheric Data Centre (NCAS BADC)',
  'partyType': 'organisation',
  'description': "The British Atmospheric Data Centre (BADC) is the Natural Environment Research Council's (NERC) Designated Data Centre for the Atmospheric Sciences. The role of the BADC is to assist UK atmospheric researchers to locate, access and interpret atmospheric data and to ensure the long-term integrity of atmospheric data produced by NERC projects. The BADC is run by he Centre for Environmental Data Archival (CEDA).",
  'deliveryPoint': 'British Atmospheric Data Centre',
  'administrativeArea': 'STFC Rutherford Appleton Laboratory',
  'city': 'Harwell Oxford',
  'country': 'UK',
  'postalCode': 'OX11 0QX',
  'electronicEmailAddress': 'badc@rl.ac.uk',
  'phone': '+44(0)1235 446432',
  'onlineResource': 'http://badc.rl.ac.uk/'},
 'role': 'metadata_owner'}